In [1]:
from typing import Dict, List, Tuple, Any, Optional
from pydantic import BaseModel, field_validator, ConfigDict
import numpy as np
from itertools import product

# ---------------------------
# Estructuras básicas de datos
# ---------------------------
class Pagos(BaseModel):
    valores: Dict[str, float]  # Jugador -> Pago
    
    model_config = ConfigDict(frozen=True)

class Estrategia(BaseModel):
    jugador: str
    estrategia: str
    
    model_config = ConfigDict(frozen=True)

class Estrategias(BaseModel):
    estrategias: Dict[str, str]  # Jugador -> Estrategia
    
    model_config = ConfigDict(frozen=True)

# ---------------------------
# Estructura principal del juego
# ---------------------------
class EstructuraDeJuego(BaseModel):
    jugadores: List[str]
    estrategias: Dict[str, List[str]]  # Jugador -> Lista de estrategias
    matriz_pagos: Dict[Tuple[str, ...], Pagos]  # Combinación de estrategias -> Pagos
    
    @field_validator('matriz_pagos')
    def validar_matriz_pagos(cls, v, values):
        estrategias = values.data.get('estrategias', {})
        jugadores = values.data.get('jugadores', [])
        
        # Verificar que todas las combinaciones sean válidas
        for combinacion in v.keys():
            if len(combinacion) != len(jugadores):
                raise ValueError("La combinación de estrategias no coincide con el número de jugadores")
            
            for jugador, estrategia in zip(jugadores, combinacion):
                if estrategia not in estrategias[jugador]:
                    raise ValueError(f"Estrategia inválida {estrategia} para jugador {jugador}")
        
        return v
    
    @field_validator('estrategias')
    def validar_estrategias_jugadores(cls, v, values):
        jugadores = values.data.get('jugadores', [])
        for jugador in jugadores:
            if jugador not in v:
                raise ValueError(f"Faltan estrategias para el jugador {jugador}")
        return v

# ---------------------------
# Funciones de utilidad
# ---------------------------
def pagos_estrategia(juego: EstructuraDeJuego, estrategias: Estrategias) -> Pagos:
    combinacion = tuple(estrategias.estrategias[j] for j in juego.jugadores)
    return juego.matriz_pagos[combinacion]

# ---------------------------
# Algoritmo de Equilibrio de Nash (versión simplificada)
# ---------------------------
def equilibrio_nash(juego: EstructuraDeJuego) -> List[Tuple[Estrategias, Pagos]]:
    equilibrios = []
    
    # Generar todas las posibles combinaciones de estrategias puras
    estrategias_combinadas = product(*[juego.estrategias[j] for j in juego.jugadores])
    
    for combinacion in estrategias_combinadas:
        es_equilibrio = True
        pagos_actual = juego.matriz_pagos[combinacion].valores
        
        for jugador in juego.jugadores:
            # Obtener índice del jugador
            idx = juego.jugadores.index(jugador)
            
            # Evaluar todas las posibles desviaciones del jugador
            for estrategia_alternativa in juego.estrategias[jugador]:
                if estrategia_alternativa == combinacion[idx]:
                    continue
                
                # Crear nueva combinación con estrategia alternativa
                nueva_combinacion = list(combinacion)
                nueva_combinacion[idx] = estrategia_alternativa
                nuevos_pagos = juego.matriz_pagos[tuple(nueva_combinacion)].valores
                
                # Verificar si la desviación es beneficiosa
                if nuevos_pagos[jugador] > pagos_actual[jugador]:
                    es_equilibrio = False
                    break
            
            if not es_equilibrio:
                break
        
        if es_equilibrio:
            estrategias_dict = {j: s for j, s in zip(juego.jugadores, combinacion)}
            equilibrios.append((
                Estrategias(estrategias=estrategias_dict),
                Pagos(valores=pagos_actual)
            ))
    
    return equilibrios


In [3]:
# ---------------------------
# Ejemplo de uso: Matching Pennies
# ---------------------------
if __name__ == "__main__":
    # Definir el juego
    matching_pennies = EstructuraDeJuego(
        jugadores=["A", "B"],
        estrategias={
            "A": ["C", "X"],
            "B": ["C", "X"]
        },
        matriz_pagos={
            ("C", "C"): Pagos(valores={"A": 0, "B": 0}),
            ("C", "X"): Pagos(valores={"A": -1, "B": 1}),
            ("X", "C"): Pagos(valores={"A": 1, "B": -1}),
            ("X", "X"): Pagos(valores={"A": -10, "B": -10}),
        }
    )
    
    # Encontrar equilibrios de Nash
    equilibrios = equilibrio_nash(matching_pennies)
    
    print("Equilibrios de Nash encontrados:")
    for eq in equilibrios:
        print(f"Estrategias: {eq[0].estrategias}")
        print(f"Pagos: {eq[1].valores}")
        print("---")

Equilibrios de Nash encontrados:
Estrategias: {'A': 'C', 'B': 'X'}
Pagos: {'A': -1.0, 'B': 1.0}
---
Estrategias: {'A': 'X', 'B': 'C'}
Pagos: {'A': 1.0, 'B': -1.0}
---


# Juego 1: La Carrera Tecnológica

**Storytelling:**  
Dos empresas de inteligencia artificial (NeuraTech y DeepMindX) deben elegir entre:

- **Estrategia A:** Invertir en I+D para un avance revolucionario (alto riesgo/recompensa)
- **Estrategia B:** Mejorar productos existentes (bajo riesgo/recompensa moderada)

**Reglas:**

- **Si ambas eligen I+D:**  
  60% de probabilidad de éxito  
  (recompensa: +8 cada una si tienen éxito, -4 si fallan)
- **Si una elige I+D y otra no:**  
  La innovadora captura el mercado  
  (recompensa: +10 para la que invierte vs -2)
- **Si ambas mejoran productos:**  
  +3 cada una


In [15]:
carrera_tecnologica = EstructuraDeJuego(
    jugadores=["NeuraTech", "DeepMindX"],
    estrategias={
        "NeuraTech": ["I+D", "Mejorar"],
        "DeepMindX": ["I+D", "Mejorar"]
    },
    matriz_pagos={
        ("I+D", "I+D"): Pagos(valores={"NeuraTech": 5, "DeepMindX": 5}),  # Valor esperado
        ("I+D", "Mejorar"): Pagos(valores={"NeuraTech": 10, "DeepMindX": -2}),
        ("Mejorar", "I+D"): Pagos(valores={"NeuraTech": -2, "DeepMindX": 10}),
        ("Mejorar", "Mejorar"): Pagos(valores={"NeuraTech": 3, "DeepMindX": 3}),
    }
)
# Encontrar equilibrios de Nash
equilibrios = equilibrio_nash(carrera_tecnologica)

print("Equilibrios de Nash encontrados:")
for eq in equilibrios:
    print(f"Estrategias: {eq[0].estrategias}")
    print(f"Pagos: {eq[1].valores}")
    print("---")

Equilibrios de Nash encontrados:
Estrategias: {'NeuraTech': 'I+D', 'DeepMindX': 'I+D'}
Pagos: {'NeuraTech': 5.0, 'DeepMindX': 5.0}
---


Prefieren invertir a quedarse afuera del mercado 


# Juego 2: El Dilema del Arquitecto

**Storytelling:**  
Dos arquitectos compiten por un proyecto urbano. Cada uno puede:

- **Cooperar:** Diseño ecológico integrado con el entorno (+2 reputación)
- **Competir:** Propuesta maximalista para impresionar al jurado  
  (+5 si gana, -3 si pierde)

**Reglas:**

- **Si ambos cooperan:**  
  Comparten el proyecto (+2 cada uno)
- **Si uno compite y otro coopera:**  
  El competitivo gana  
  (recompensa: +5 para el que compite vs -1 por daño reputacional para el cooperador)
- **Si ambos compiten:**  
  50% de ganar  
  (resultado: +5/-3) con valor esperado de +1 para cada uno

In [16]:
dilema_arquitecto = EstructuraDeJuego(
    jugadores=["Arquitecto1", "Arquitecto2"],
    estrategias={
        "Arquitecto1": ["Cooperar", "Competir"],
        "Arquitecto2": ["Cooperar", "Competir"]
    },
    matriz_pagos={
        ("Cooperar", "Cooperar"): Pagos(valores={"Arquitecto1": 2, "Arquitecto2": 2}),
        ("Cooperar", "Competir"): Pagos(valores={"Arquitecto1": -1, "Arquitecto2": 5}),
        ("Competir", "Cooperar"): Pagos(valores={"Arquitecto1": 5, "Arquitecto2": -1}),
        ("Competir", "Competir"): Pagos(valores={"Arquitecto1": 1, "Arquitecto2": 1}),
    }
)

# Encontrar equilibrios de Nash
equilibrios = equilibrio_nash(dilema_arquitecto)

print("Equilibrios de Nash encontrados:")
for eq in equilibrios:
    print(f"Estrategias: {eq[0].estrategias}")
    print(f"Pagos: {eq[1].valores}")
    print("---")

Equilibrios de Nash encontrados:
Estrategias: {'Arquitecto1': 'Competir', 'Arquitecto2': 'Competir'}
Pagos: {'Arquitecto1': 1.0, 'Arquitecto2': 1.0}
---


Parecido al dilema del prisionera que aunque convenga cooperar ambos se traicionan y pierden 

# Juego 3: La Guerra de Contenidos

**Storytelling:**  
Dos streamers (LunaGamer y PixelKing) deciden su estrategia de contenido:

- **Calidad:** Videos editados profesionalmente (alto costo/alta retención)
- **Cantidad:** Streams diarios largos (bajo costo/audiencia casual)

**Pagos:**

- **Si ambos eligen Calidad:**  
  Audiencia se divide (+4 cada uno)
- **Si uno elige Calidad y otro Cantidad:**  
  El de Cantidad captura más viewers (+6 vs +2)
- **Si ambos eligen Cantidad:**  
  Saturación del mercado (+1 cada uno)

In [17]:
guerra_contenidos = EstructuraDeJuego(
    jugadores=["LunaGamer", "PixelKing"],
    estrategias={
        "LunaGamer": ["Calidad", "Cantidad"],
        "PixelKing": ["Calidad", "Cantidad"]
    },
    matriz_pagos={
        ("Calidad", "Calidad"): Pagos(valores={"LunaGamer": 4, "PixelKing": 4}),
        ("Calidad", "Cantidad"): Pagos(valores={"LunaGamer": 2, "PixelKing": 6}),
        ("Cantidad", "Calidad"): Pagos(valores={"LunaGamer": 6, "PixelKing": 2}),
        ("Cantidad", "Cantidad"): Pagos(valores={"LunaGamer": 1, "PixelKing": 1}),
    }
)

# Encontrar equilibrios de Nash
equilibrios = equilibrio_nash(dilema_arquitecto)

print("Equilibrios de Nash encontrados:")
for eq in equilibrios:
    print(f"Estrategias: {eq[0].estrategias}")
    print(f"Pagos: {eq[1].valores}")
    print("---")

Equilibrios de Nash encontrados:
Estrategias: {'Arquitecto1': 'Competir', 'Arquitecto2': 'Competir'}
Pagos: {'Arquitecto1': 1.0, 'Arquitecto2': 1.0}
---


Se prioriza cantidad sobre calidad
